In [ ]:
#!pip install youtube_transcript_api
#!pip isntall pandasql 

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
from pandasql import sqldf 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [6]:
pysqldf = lambda q: sqldf(q, globals())

In [7]:
stop = stopwords.words('english')

In [8]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [9]:
links = [
    'BDqvzFY72mg',
    'f5nbT4xQqwI',
    's48b9B5gd88',
    '4eUS8trd_yI',
    'aKW_Vsk4hzs',
    'q53DF6ySOZg',
    'T3-VlQu3iRM'
]

In [ ]:
transcripts = []
for v in links:
    try:
        df = pd.DataFrame(YouTubeTranscriptApi.get_transcript(v))
        df['video_id'] = v
        transcripts.append(df)
        print(v, 'translated')
    except:
        print(v, 'failed to translate')

BDqvzFY72mg translated
f5nbT4xQqwI translated
s48b9B5gd88 translated
4eUS8trd_yI translated


In [ ]:
df_transcripts = pd.concat(transcripts).reset_index(drop=True)

In [ ]:
df_transcripts['alpha_text'] = df_transcripts.text.str.replace("[^a-zA-Z]", ' ')

In [ ]:
# remove stop words

df_transcripts['no_stop'] = df_transcripts['alpha_text'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word.lower() not in stop))

In [ ]:
#df_transcripts

In [ ]:
df_transcripts['lemmatized_text'] = df_transcripts['no_stop'].apply(
    lambda words: ' '.join(lemmatizer.lemmatize(w) for w in words.split()))

In [ ]:
df_transcripts['row_num'] = df_transcripts.index

In [ ]:
df_transcripts

In [ ]:
df_transcripts.to_csv('transcripts.csv', index=False)

In [ ]:
## The number of cells that will be squashed into a single cell is set by the 
## ranking - for instance, to combine every 10 cells, add 9 (to offset index at 1)
## and divide by 10 (keeping only the integer remainder)

df_transcripts_m10 = pysqldf("""
WITH df_ranked AS

(SELECT
    *,
    (RANK () OVER ( 
        PARTITION BY video_id
        ORDER BY start ASC
    ) + 9)/ 10 RNK 
FROM
    df_transcripts
ORDER BY row_num
)

SELECT 
    GROUP_CONCAT(text, ' ') as text,
    GROUP_CONCAT(alpha_text, ' ') as alpha_text,
    GROUP_CONCAT(no_stop, ' ') as no_stop,
    GROUP_CONCAT(lemmatized_text, ' ') as lemmatized_text,
    MIN(start), 
    MAX(start), 
    SUM(duration), 
    video_id 
FROM df_ranked
GROUP BY
    rnk, video_id
ORDER BY video_id, MIN(start)
""")

In [ ]:
df_transcripts_m10.to_csv('transcripts_m10.csv', index=False) 